바이오메디컬인공지능 개인과제6

바이오메디컬공학부 2018----- 손무현

## Sequence-to-sequence model로 언어 번역하기


- https://tutorials.pytorch.kr/beginner/torchtext_translation.html

In [1]:
%matplotlib inline
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 8.1MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907055 sha256=d2c565b0447c641e31f3d6b3d5a41a414ad2115f33fee5830024cf05cda8c1bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-evp8_lrl/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [2]:
import math
import torchtext
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
from torch import Tensor
import io
import time

In [5]:
import spacy
en_tokenizer = get_tokenizer('spacy', language='en')
de_tokenizer = get_tokenizer('spacy', language='de')

### 텍스트파일의 전처리
- 아래의 코드는 text file을 전처리하는 과정을 담고 있으나, 실행시간이 길어 미리 처리한 파일을 불러오도록 주석처리를 하였습니다.
- 다른 text파일을 활용하여 실습을 진행하고 싶은 경우, 아래의 코드를 이해하여 전처리 과정을 수행하여야 합니다.

In [6]:
'''

url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      counter.update(tokenizer(string_))
  return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

def data_process(filepaths):
  raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                            dtype=torch.long)
    data.append((de_tensor_, en_tensor_))
  return data


de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)

'''

'\n\nurl_base = \'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/\'\ntrain_urls = (\'train.de.gz\', \'train.en.gz\')\nval_urls = (\'val.de.gz\', \'val.en.gz\')\ntest_urls = (\'test_2016_flickr.de.gz\', \'test_2016_flickr.en.gz\')\n\ntrain_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]\nval_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]\ntest_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]\n\ndef build_vocab(filepath, tokenizer):\n  counter = Counter()\n  with io.open(filepath, encoding="utf8") as f:\n    for string_ in f:\n      counter.update(tokenizer(string_))\n  return Vocab(counter, specials=[\'<unk>\', \'<pad>\', \'<bos>\', \'<eos>\'])\n\ndef data_process(filepaths):\n  raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))\n  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))\n  data = []\n  for (raw_de, raw_en) in zip

## 학습에 사용할 Data를 가져오기
- 미리 만들어 놓은 vocab, tokenized data를 불러옵니다.
- 실습데이터를 올려놓은 구글드라이브 경로를 설정하여 가져옵니다.

In [7]:
path0 = '/content/drive/MyDrive/de2en/'
de_vocab=torch.load(path0+'de_vocab.pt')
en_vocab=torch.load(path0+'en_vocab.pt')
train_data = torch.load(path0+'train_data.pt')
val_data = torch.load(path0+'val_data.pt')

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 모델 및 학습을 위한 함수들의 정의

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
BATCH_SIZE = 64
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return en_batch, de_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)

In [11]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)

class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
                
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [12]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + 
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [13]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [40]:
SRC_VOCAB_SIZE = len(en_vocab)
TGT_VOCAB_SIZE = len(de_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 10

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, 
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.000011, betas=(0.9, 0.98), eps=1e-9
)

In [41]:
def train_epoch(model, train_iter, optimizer):
  model.train()
  losses = 0
  for idx, (src, tgt) in enumerate(train_iter):
      src = src.to(device)
      tgt = tgt.to(device)
            
      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)
      
      optimizer.zero_grad()
      
      tgt_out = tgt[1:,:]
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
      loss.backward()

      optimizer.step()
      losses += loss.item()
  return losses / len(train_iter)


def evaluate(model, val_iter):
  model.eval()
  losses = 0
  for idx, (src, tgt) in (enumerate(valid_iter)):
    src = src.to(device)
    tgt = tgt.to(device)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,
                              src_padding_mask, tgt_padding_mask, src_padding_mask)
    tgt_out = tgt[1:,:]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    losses += loss.item()
  return losses / len(val_iter)

In [42]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys


def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
  model.eval()
  tokens = [BOS_IDX] + [src_vocab.stoi[tok] for tok in src_tokenizer(src)]+ [EOS_IDX]
  num_tokens = len(tokens)
  src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
  return " ".join([tgt_vocab.itos[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

학습이 진행되기 전에 결과를 확인해 봅니다.

In [24]:
translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu .", en_vocab, de_vocab, en_tokenizer)

' Bühnenmitte Fußböden backen backen backen backen backen backen Bewässerungsschlauch Bühnenmitte Fußböden backen backen backen backen'

## 모델을 학습시키기

In [47]:
for epoch in range(1, NUM_EPOCHS+1):
  start_time = time.time()
  train_loss = train_epoch(transformer, train_iter, optimizer)
  end_time = time.time()
  val_loss = evaluate(transformer, valid_iter)
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 2.286, Val loss: 2.510, Epoch time = 49.139s
Epoch: 2, Train loss: 2.251, Val loss: 2.478, Epoch time = 48.451s
Epoch: 3, Train loss: 2.217, Val loss: 2.453, Epoch time = 48.824s
Epoch: 4, Train loss: 2.186, Val loss: 2.434, Epoch time = 48.669s
Epoch: 5, Train loss: 2.155, Val loss: 2.407, Epoch time = 48.761s
Epoch: 6, Train loss: 2.125, Val loss: 2.378, Epoch time = 48.655s
Epoch: 7, Train loss: 2.094, Val loss: 2.366, Epoch time = 48.673s
Epoch: 8, Train loss: 2.067, Val loss: 2.349, Epoch time = 48.752s
Epoch: 9, Train loss: 2.042, Val loss: 2.323, Epoch time = 48.730s
Epoch: 10, Train loss: 2.013, Val loss: 2.307, Epoch time = 48.647s


In [ ]:
#torch.save(transformer.state_dict(),path0+'en2de.pth')

In [ ]:
#transformer.load_state_dict(torch.load(path0+'en2de.pth'))

## 학습이 완료된 모델을 활용하여 독일어에 대한 영어번역결과를 확인하기

In [48]:
translate(transformer, "I am a boy", en_vocab, de_vocab, en_tokenizer)

' Ein Surfer , der einen Jungen auf der Junge klettert'

- 최종 모델에 사용한 Hyperparameter의 값들

EMB_SIZE = 512

NHEAD = 8

FFN_HID_DIM = 512

BATCH_SIZE = 64

NUM_ENCODER_LAYERS = 3

NUM_DECODER_LAYERS = 3

NUM_EPOCHS = 10

lr=0.000011

BATCH_SIZE와 인코더 디코더의 layer number, 그리고 NHEAD를 변경해가며 training을 시도했지만 loss값이 적절하게 나오지 않아 learning rate를 변경해가며 parameter를 update하였습니다.

- Training 과정에서의 epoch별 loss 변화

Epoch: 1, Train loss: 2.286, Val loss: 2.510, Epoch time = 49.139s

Epoch: 2, Train loss: 2.251, Val loss: 2.478, Epoch time = 48.451s

Epoch: 3, Train loss: 2.217, Val loss: 2.453, Epoch time = 48.824s

Epoch: 4, Train loss: 2.186, Val loss: 2.434, Epoch time = 48.669s

Epoch: 5, Train loss: 2.155, Val loss: 2.407, Epoch time = 48.761s

Epoch: 6, Train loss: 2.125, Val loss: 2.378, Epoch time = 48.655s

Epoch: 7, Train loss: 2.094, Val loss: 2.366, Epoch time = 48.673s

Epoch: 8, Train loss: 2.067, Val loss: 2.349, Epoch time = 48.752s

Epoch: 9, Train loss: 2.042, Val loss: 2.323, Epoch time = 48.730s

Epoch: 10, Train loss: 2.013, Val loss: 2.307, Epoch time = 48.647s

- 비교적 번역이 괜찮은 문장을 최소 2개 이상 (구글번역과 결과 비교)

In [39]:
translate(transformer, "I am a boy", en_vocab, de_vocab, en_tokenizer)

' Ein Arzt rollt einen Jungen . \n '

In [37]:
translate(transformer, "it is an apple", en_vocab, de_vocab, en_tokenizer)

' Ein Hügel befindet sich einem Apfel . \n '

- 번역이 매우 좋지 않은 문장을 최소 2개 이상 (구글번역과 결과 비교)

In [36]:
translate(transformer, "If you don't risk anything you risk even more.", en_vocab, de_vocab, en_tokenizer)

' Das Bild , die wie man ein Schäferhund unter dem Fell machen , was es schneit .'

In [29]:
translate(transformer, " Attempt easy tasks as if they were difficult, and difficult as if they were easy; in the one case that confidence may not fall asleep, in the other that it may not be dismayed.", en_vocab, de_vocab, en_tokenizer)

' Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein Ein'